In [237]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px

import inflection

In [238]:
#========================
#   0.1. Helper Variables
#========================

In [239]:
RAW_DATA_PATH = f"zomato.csv"

In [240]:
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

In [241]:
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}

In [242]:
#========================
#   0.2. Helper Functions
#========================

In [243]:
def show_dataframe_dimensions(dataframe):
    """ Mostra as dimensões do dataframe
    
    """
    print(f"Number of Rows: {dataframe.shape[0]}")
    print(f"Number of Columns: {dataframe.shape[1]}")
    

In [244]:
def rename_columns(dataframe):
    """  Esta função tem a responsabilidade de limpar o dataframe
    
    Tipos de limpeza: 
    1. Remoção dos espaços das variáveis de texto
    2. Mudança de nome das colunas
    
    Input: Dataframe
    Output: Dataframe       
    """
    
    df = dataframe.copy()

    title = lambda x: inflection.titleize(x)

    snakecase = lambda x: inflection.underscore(x)

    spaces = lambda x: x.replace(" ", "")

    cols_old = list(df.columns)

    cols_old = list(map(title, cols_old))

    cols_old = list(map(spaces, cols_old))

    cols_new = list(map(snakecase, cols_old))

    df.columns = cols_new

    return df

In [245]:
def country_name(country_id):
    """ Troca o código por nomes na coluna paises    """
    return COUNTRIES[country_id]

In [246]:
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

In [247]:
def color_name(color_code):
    return COLORS[color_code]

In [248]:
def adjust_columns_order(dataframe):
    df = dataframe.copy()

    new_cols_order = [
        "restaurant_id",
        "restaurant_name",
        "country",
        "city",
        "address",
        "locality",
        "locality_verbose",
        "longitude",
        "latitude",
        "cuisines",
        "price_type",
        "average_cost_for_two",
        "currency",
        "has_table_booking",
        "has_online_delivery",
        "is_delivering_now",
        "aggregate_rating",
        "rating_color",
        "color_name",
        "rating_text",
        "votes",
    ]

    return df.loc[:, new_cols_order]

In [249]:
def process_data(file_path):
    """  Esta função tem a responsabilidade de modificar  o dataframe
    
    Tipos de limpeza: 
    1. Criação de uma coluna com os nomes dos paises
    2. Criação de uma coluna de categoria de comida 
    3. Criação uma coluna com o nome das Cores
 
    
    Input: Dataframe
    Output: Dataframe       
    
    """
    
    df = pd.read_csv(file_path)

    df = df.dropna()

    df = rename_columns(df)
    
    #Criação de uma coluna com os nomes dos paises de acordo com os códigos
    df["country"] = df.loc[:, "country_code"].apply(lambda x: country_name(x))
    
    #Criação de uma coluna de categoria de comida 
    df["price_type"] = df.loc[:, "price_range"].apply(lambda x: create_price_tye(x))
    
    #Criação do nome das Cores
    df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: color_name(x))

    df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])
    
    
    df = df.drop_duplicates()

    df = adjust_columns_order(df)

    df.to_csv("zomato.csv", index=False)

    return df

In [250]:
#========================
#   0.3. Load Data
#========================

In [224]:
df_raw = pd.read_csv(RAW_DATA_PATH)
df_raw.head()

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,rating_color,color_name,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,1100,Botswana Pula(P),1,0,0,4.6,3F7E00,darkgreen,Excellent,619
1,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,3100,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,469
2,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,800,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,867
3,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,700,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,858
4,6304833,Manam,Philippines,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,...,700,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,930


In [225]:
#========================
#   1. DATA DESCRIPTION
#========================

In [251]:
df1 = df_raw.copy()
df1.head()

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,rating_color,color_name,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,1100,Botswana Pula(P),1,0,0,4.6,3F7E00,darkgreen,Excellent,619
1,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,3100,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,469
2,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,800,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,867
3,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,700,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,858
4,6304833,Manam,Philippines,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,...,700,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,930


In [227]:
#========================
#   1.1 Columns Description
#========================

In [228]:
#========================
#   1.2 Rename Columns
#========================

In [282]:
df1 = rename_columns(df1)
df1.head(1)

,restaurant,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,rating_color,color_name,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,1100,Botswana Pula(P),1,0,0,4.6,3F7E00,darkgreen,Excellent,619


In [230]:
#========================
#   1.4. Data Types
#========================

In [253]:
df1.dtypes

restaurant                int64
restaurant_name          object
country                  object
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
price_type               object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
aggregate_rating        float64
rating_color             object
color_name               object
rating_text              object
votes                     int64
dtype: object

In [232]:
#========================
#   1.5. Check NA and Treat NA
#========================

In [254]:
df1.isna().sum()
df1 = df1.dropna()
df1.isna().sum()

restaurant              0
restaurant_name         0
country                 0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
price_type              0
average_cost_for_two    0
currency                0
has_table_booking       0
has_online_delivery     0
is_delivering_now       0
aggregate_rating        0
rating_color            0
color_name              0
rating_text             0
votes                   0
dtype: int64

In [234]:
#========================
#   2. Answers
#========================

In [236]:
#df2 = process_data(RAW_DATA_PATH)
#df2.head(1)

KeyError: 'country_code'

In [235]:
# -------------------- Geral 

In [258]:
restaurant_uniques = len(df1.loc[:, 'restaurant'].unique())
restaurant_uniques

# df2.shape[0] ---> meigaron

6929

In [259]:
countries_uniques = len(df1.loc[:, 'country'].unique())
countries_uniques

15

In [260]:
cities_uniques = len(df1.loc[:, 'city'].unique())
cities_uniques


# df2.loc[:, 'city'].nunique()

125

In [261]:
df1.loc[:, 'votes'].sum()

4194533

In [264]:
df1.loc[:, 'cuisines'].nunique()

165

In [ ]:
# --------------- Pais

In [ ]:
df_aux = (df1.loc[:, ['city', 'country']]
                 .groupby('country')
                 .nunique()
                 .sort_values('city', ascending=False)
                 .reset_index()
                 .iloc[0, 0])

df_aux

In [279]:
df_aux = (df1.loc[:, ['restaurant', 'country']]
                 .groupby('country')
                 .nunique()
                 .sort_values('restaurant', ascending=False)
                 .reset_index()
                 .iloc[0, 0])

df_aux

'India'

In [286]:
(df1.loc[df1['aggregate_rating'] >= 4, ['restaurant', 'country']]
                                         .groupby('country')
                                         .count()
                                         .sort_values('restaurant', ascending=False)
                                         .reset_index()).iloc[0, 0]

'India'

In [287]:
df_aux = (df1.loc[:, ['cuisines', 'country']]
                 .groupby('country')
                 .nunique()
                 .sort_values('cuisines', ascending=False)
                 .reset_index()
                 .iloc[0, 0])

df_aux

'India'

In [288]:
df_aux = (df1.loc[:, ['votes', 'country']]
                 .groupby('country')
                 .nunique()
                 .sort_values('votes', ascending=False)
                 .reset_index()
                 .iloc[0, 0])

df_aux

'India'

In [292]:
(df1.loc[df1['is_delivering_now'] == 1, ['restaurant', 'country']]
                                         .groupby('country')
                                         .count()
                                         .sort_values('restaurant', ascending=False)
                                         .reset_index()).iloc[0, 0]




'India'

In [293]:
(df1.loc[df1['has_table_booking'] == 1, ['restaurant', 'country']]
                                         .groupby('country')
                                         .count()
                                         .sort_values('restaurant', ascending=False)
                                         .reset_index()).iloc[0, 0]

'India'

In [296]:
(df1.loc[:, ['country', 'votes']]
        .groupby('country')
        .mean()
        .sort_values('votes', ascending=False)
        .reset_index()).iloc[0, 0]


'Indonesia'

In [297]:
(df1.loc[:, ['country', 'aggregate_rating']]
        .groupby('country')
        .mean()
        .sort_values('aggregate_rating', ascending=False)
        .reset_index()).iloc[0, 0]


'Indonesia'

In [298]:
(df1.loc[:, ['country', 'aggregate_rating']]
        .groupby('country')
        .mean()
        .sort_values('aggregate_rating', ascending=True)
        .reset_index()).iloc[0, 0]


'Brazil'

In [300]:
(df1.loc[:, ['country', 'average_cost_for_two']]
        .groupby('country')
        .mean()
        .sort_values('average_cost_for_two', ascending=False)
        .reset_index())


,country,average_cost_for_two
0,Indonesia,303000.000000
1,Australia,138959.783333
2,Sri Lanka,2579.375000
3,Philippines,1227.825000
4,India,704.400514
5,South Africa,339.228324
6,Qatar,174.000000
7,United Arab Emirates,153.716667
8,Singapure,141.437500
9,Brazil,138.812500


In [ ]:
# --------------- Cidade

In [302]:
(df1.loc[:, ['city', 'restaurant']]
        .groupby('city')
        .count()
        .sort_values('restaurant', ascending=False)
        .reset_index()).iloc[0, 0]

'Abu Dhabi'

In [303]:
(df1.loc[df1['aggregate_rating'] >= 4, ['restaurant', 'city']]
                                         .groupby('city')
                                         .count()
                                         .sort_values('restaurant', ascending=False)
                                         .reset_index()).iloc[0, 0]

'London'

In [304]:
(df1.loc[df1['aggregate_rating'] <= 2.5, ['restaurant', 'city']]
                                         .groupby('city')
                                         .count()
                                         .sort_values('restaurant', ascending=False)
                                         .reset_index()).iloc[0, 0]

'Gangtok'

In [316]:
(df1.loc[:, ['city', 'average_cost_for_two']]
        .groupby('city')
        .mean()
        .sort_values('average_cost_for_two', ascending=False)
        .reset_index()).iloc[0, 0]

'Adelaide'

In [319]:
(df1.loc[:, ['city', 'cuisines']]
        .groupby('city')
        .nunique()
        .sort_values('cuisines', ascending=False)
        .reset_index()).iloc[0, 0]

'Birmingham'

In [320]:
(df1.loc[df1['has_table_booking'] == 1, ['city', 'restaurant']]
                                        .groupby('city')
                                        .count()
                                        .sort_values('restaurant', ascending=False)
                                        .reset_index()).iloc[0, 0]

'Bangalore'

In [321]:
(df1.loc[df1['is_delivering_now'] == 1, ['city', 'restaurant']]
                                        .groupby('city')
                                        .count()
                                        .sort_values('restaurant', ascending=False)
                                        .reset_index()).iloc[0, 0]

'Vadodara'

In [322]:
(df1.loc[df1['has_online_delivery'] == 1, ['city', 'restaurant']]
                                        .groupby('city')
                                        .count()
                                        .sort_values('restaurant', ascending=False)
                                        .reset_index()).iloc[0, 0]

'Bhopal'

In [ ]:
# --------------- Restaurantes

In [335]:
cols = ['restaurant', 'restaurant_name', 'votes']

df1.loc[:, cols].sort_values(['votes', 'restaurant'], ascending=[False, True]).iloc[0, 1]


'Bawarchi'

In [340]:
cols = ['restaurant', 'restaurant_name', 'aggregate_rating']

df1.loc[:, cols].sort_values(['aggregate_rating', 'restaurant'], ascending=[False, True]).iloc[0, 1]

'Indian Grill Room'

In [341]:
cols = ['restaurant', 'restaurant_name', 'average_cost_for_two']

df1.loc[:, cols].sort_values(['average_cost_for_two', 'restaurant'], ascending=[False, True]).iloc[0, 1]

"d'Arry's Verandah Restaurant"

In [349]:
cols = ['restaurant', 'restaurant_name', 'aggregate_rating']

df1.loc[df1['cuisines'] == 'Brazilian', cols].sort_values(['aggregate_rating', 'restaurant'], ascending=True).iloc[0, 1]

'Loca Como tu Madre'

In [354]:
cols = ['restaurant', 'restaurant_name', 'aggregate_rating']

lines = (df1['cuisines'] == 'Brazilian') & (df1['country'] == 'Brazil')

df1.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant'], ascending=[False, True]).iloc[0, 1]



'Braseiro da Gávea'

In [355]:
(df1.loc[:, ['votes', 'has_online_delivery'] ]
                                         .groupby('has_online_delivery')
                                         .mean()
                                         .sort_values('votes', ascending=False)
                                         .reset_index())

,has_online_delivery,votes
0,1,838.821664
1,0,479.421018


In [356]:
(df1.loc[:, ['average_cost_for_two', 'has_table_booking'] ]
                                         .groupby('has_table_booking')
                                         .mean()
                                         .sort_values('average_cost_for_two', ascending=False)
                                         .reset_index())

,has_table_booking,average_cost_for_two
0,1,69998.423810
1,0,3488.596866


In [357]:
lines = (df1['country'] == 'United States of America') & ((df1['cuisines'] == 'BBQ') | (df1['cuisines'] == 'Japanese'))

(df1.loc[lines, ['average_cost_for_two', 'cuisines'] ]
                                         .groupby('cuisines')
                                         .mean()
                                         .sort_values('average_cost_for_two', ascending=False)
                                         .reset_index())

,cuisines,average_cost_for_two
0,Japanese,56.406250
1,BBQ,39.642857


In [ ]:
# -------------------- Tipos de Culinária

In [363]:
(df1.loc[df1['cuisines'] == 'Italian', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=[False, True])).iloc[0, 0]

'Darshan'

In [358]:
df1.head(1)

,restaurant,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,rating_color,color_name,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,1100,Botswana Pula(P),1,0,0,4.6,3F7E00,darkgreen,Excellent,619


In [364]:
(df1.loc[df1['cuisines'] == 'Italian', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=True)).iloc[0, 0]

'Avenida Paulista'

In [365]:
(df1.loc[df1['cuisines'] == 'American', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=[False, True])).iloc[0, 0]

'Burger & Lobster'

In [366]:
(df1.loc[df1['cuisines'] == 'American', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=True)).iloc[0, 0]

'Alston Bar & Beef'

In [367]:
(df1.loc[df1['cuisines'] == 'Arabian', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=[False, True])).iloc[0, 0]

'Mandi@36'

In [368]:
(df1.loc[df1['cuisines'] == 'Arabian', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=True)).iloc[0, 0]

'Raful'

In [369]:
(df1.loc[df1['cuisines'] == 'Japanese', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=[False, True])).iloc[0, 0]

'Sushi Samba'

In [370]:
(df1.loc[df1['cuisines'] == 'Japanese', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=[True, True])).iloc[0, 0]

'Banzai Sushi'

In [371]:
(df1.loc[df1['cuisines'] == 'Home-made', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=[False, True])).iloc[0, 0]

'Kanaat Lokantası'

In [372]:
(df1.loc[df1['cuisines'] == 'Home-made', ['restaurant_name', 'restaurant', 'aggregate_rating']]
                                        .sort_values(['aggregate_rating', 'restaurant'], ascending=[True, True])).iloc[0, 0]

'GurMekan Restaurant'

In [374]:
(df1.loc[:, ['cuisines', 'average_cost_for_two']]
            .sort_values('average_cost_for_two', ascending=False)
            .iloc[0, 0])

'Modern Australian'

In [377]:
(df1.loc[:, ['cuisines', 'aggregate_rating']]
            .groupby('cuisines')
            .max()
            .sort_values('aggregate_rating', ascending=False)
            .reset_index()).iloc[0, 0]

'Indian'

In [380]:
lines = (df1['has_online_delivery'] == 1) & (df1['is_delivering_now'] == 1)

(df1.loc[lines, ['cuisines', 'restaurant']]
            .groupby('cuisines')
            .count()
            .sort_values('restaurant', ascending=False)
            .reset_index()).iloc[0, 0]

'North Indian'